## Import Packages

In [2]:
import os
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import bs4
import numpy as np

import time
import uuid
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium import webdriver

## Directories

In [5]:
SZ_STOREAGE_DIR = "../input/sz_html-seiten"

In [7]:
Zeit_STOREAGE_DIR = "../input/zeit_html-seiten"

In [9]:
SPIEGEL_STOREAGE_DIR = "../input/spiegel_html-seiten"

In [11]:
OUTPUT_DIR = "../output"

### date format

In [25]:
now_str = datetime.now().strftime("%Y-%m-%d")

## Data Extraction

### Sueddeutsche

#### HTML-Ihnalte extrahieren

In [27]:
sz = "https://www.sueddeutsche.de/thema/Nahost"
target_date = datetime.strptime("08.10.2025", "%d.%m.%Y") #Ziel-Datum für die Iteration der Elementent wählen
name = "sz"

driver = webdriver.Chrome()
driver.get(sz)

try:
    wait = WebDriverWait(driver, 10)

    iframe = wait.until(EC.presence_of_element_located((By.XPATH, "//iframe[contains(@id, 'sp_message_iframe')]")))
    print("Iframe gefunden.")

    driver.switch_to.frame(iframe)
    print("Wechsel zu Iframe")

    cookie_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@title, 'Ich bin einverstanden')]")))
    
    cookie_button.click()
    print("Cookie Akzeptiert.")
    
    driver.switch_to.default_content()
    time.sleep(2)

except TimeoutException:
    print("Kein Banner gefunden oder Fehler beim Klicken.")
    driver.switch_to.default_content()

date_selector = (By.CSS_SELECTOR, 'span.css-0')

while True:
    try:
        count_before = len(driver.find_elements(*date_selector))
    
        wait = WebDriverWait(driver, 10)
        load_element = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "css-1kk8krp"))
        )

        driver.execute_script("arguments[0].scrollIntoView(true);", load_element)
        time.sleep(0.5) 

        driver.execute_script("arguments[0].click();", load_element)

        try:
            wait.until(
                lambda d: len(d.find_elements(*date_selector)) > count_before
            )
        except TimeoutException:
            break 
            
        all_dates = driver.find_elements(*date_selector)
        if not all_dates:
            break

        last_date_text = all_dates[-1].text.strip()

        try:
            current_date = datetime.strptime(last_date_text, "%d.%m.%Y")
            if current_date < target_date:
                break
        except ValueError: 
            pass 

    except TimeoutException:
        break

rohes_html = driver.page_source

driver.quit()

file_name = os.path.join(
    SZ_STOREAGE_DIR,
    f"{now_str}-{name}.html",
)

try:
    with open(file_name, "w", encoding="utf-8") as f:
        f.write(rohes_html)
    print(f"Erfolgreich gespeichert in: {file_name}")

except Exception as e:
    print(f"Fehler beim Speichern der Datei: {e}")

Iframe gefunden.
Wechsel zu Iframe
Cookie Akzeptiert.
Erfolgreich gespeichert in: ../input/sz_html-seiten/2026-01-30-sz.html


### Daten abfangen

In [31]:
article_id = []
title = []
content_preview = []
datum = []

sz="https://www.sueddeutsche.de/thema/Nahost"

sz_html = os.path.join(f"../input/sz_html-seiten/{datetime.now().strftime('%Y-%m-%d')}-sz.html")

with open(sz_html, "r", encoding= "utf-8") as f:
    sz_text = f.read()

sz_soup = BeautifulSoup(sz_text, "html.parser")
sz_article_tag = sz_soup.select('article.sz-teaserlist-element')

for article in sz_article_tag:
    
    id_tag = article.attrs.get('id', 'KEINE ID GEFUNDEN')
    article_id.append(id_tag)

    #Titel des Artikels
#------------------------------------------------------------------------------------------------------    
    title_tag = article.select('[data-manual="teaser-title"]')

    if title_tag:

        for t in title_tag:
        
            title.append(t.get_text(strip=True))
                    
    else: 
        title.append('NA')

    # Vorschau des Artikel Inhaltes
#------------------------------------------------------------------------------------------------------    
    preview_tag = article.select('[data-manual="teaser-text"]')

    if preview_tag:
        for p in preview_tag:
        
            content_preview.append(p.get_text(strip=True))

    else:
        content_preview.append('NA')

    # Datum der Veröffentlichung
#------------------------------------------------------------------------------------------------------    
    datum_tag = article.select('span.css-0')

    
    if datum_tag:

        for d in datum_tag:
            
            datum.append(d.get_text(strip=True))

    else:
        datum.append('NA')


print(article_id)
print(title)
print(content_preview)
print(datum)

['teaser-httpswwwsueddeutschedepolitiksyrien-kurden-sdf-einigung-waffenstillstand-li3378799', 'teaser-httpswwwsueddeutschedepolitiksyrien-damaskus-und-kurden-beschliessen-waffenstillstand-li3378846', 'teaser-httpswwwsueddeutschedemeinungiran-proteste-exilgemeinde-kommentar-li3376141', 'teaser-httpswwwsueddeutschedepolitikisrael-auch-die-armee-geht-von-70-000-toten-in-gaza-aus-li3378849', 'teaser-httpswwwsueddeutschedepolitiknahost-israel-will-gaza-grenze-nach-aegypten-oeffnen-li3378843', 'teaser-httpswwwsueddeutschedepolitikbeirut-hotel-commodore-li3375171', 'teaser-httpswwwsueddeutschedepolitikiran-massenverhaftungen-regierungsgegnern-proteste-revolutionsgarde-botschafter-li3378340', 'teaser-httpswwwsueddeutschedepolitikpodcast-nachrichten-auf-den-punkt-syrien-kurden-li3377932', 'teaser-httpswwwsueddeutschedepolitikeu-iran-revolutionsgarde-terrorliste-li3377895', 'teaser-httpswwwsueddeutschedepolitikgazastreifen-medien-zugangssperre-klage-journalisten-li3377888', 'teaser-httpswwwsuedd

In [33]:
article_dict = {
    'id': article_id,
    'title': title,
    'preview': content_preview,
    'date': datum
}

In [35]:
sz_df = pd.DataFrame(article_dict)
sz_df["zeitschrift"] = "sz"
sz_df

,id,title,preview,date,zeitschrift
0,teaser-httpswwwsueddeutschedepolitiksyrien-kur...,Einigung zwischen Kurden und Regierung,Die SDF einigen sich mit der syrischen Regieru...,Heute 15:29 Uhr,sz
1,teaser-httpswwwsueddeutschedepolitiksyrien-dam...,Damaskus und Kurden beschließen Waffenstillstand,Die kurdisch angeführten Syrischen Demokratisc...,Heute 15:21 Uhr,sz
2,teaser-httpswwwsueddeutschedemeinungiran-prote...,So verraten die Exil-Iraner ihr eigenes Volk,"Sie sind zerstritten, ja verfeindet. Dabei bra...",Heute 15:16 Uhr,sz
3,teaser-httpswwwsueddeutschedepolitikisrael-auc...,Auch die Armee geht von 70 000 Toten in Gaza aus,Israels Militär geht israelischen Medienberich...,Heute 15:15 Uhr,sz
4,teaser-httpswwwsueddeutschedepolitiknahost-isr...,Israel will Gaza-Grenze nach Ägypten öffnen,Israel will im Gazastreifen den nach Ägypten f...,Heute 15:07 Uhr,sz
...,...,...,...,...,...
345,teaser-httpswwwsueddeutschedeprojekteartikelpo...,Niemand bleibt unberührt,Seit zwei Jahren lassen Zigtausende Männer ihr...,6.10.2025,sz
346,teaser-httpswwwsueddeutschedepolitikgaza-hams-...,Hamas-Geiseln: 730 Tage Martyrium,Zwei Jahre nach dem Überfall der Hamas sind wo...,6.10.2025,sz
347,teaser-httpswwwsueddeutschedepolitikisrael-ham...,Israel und Hamas verhandeln über Kriegsende,Auf Grundlage des US-Friedensplans sprechen be...,6.10.2025,sz
348,teaser-httpswwwsueddeutschedemeinungesc-israel...,Es ist doch nur ein Gesangswettbewerb,NA,6.10.2025,sz


### Zeit

### Navigation der HTML-Seite

In [37]:
zeit = "https://www.zeit.de/thema/israel"
name = "zeit"

driver = webdriver.Chrome()
driver.get(zeit)

#---------------------Cookie-Button anklicken------------------------------------------------
try:
    wait = WebDriverWait(driver, 10) #Wartezeit einrichten

    iframes = driver.find_elements(By.XPATH, "//iframe[contains(@id, 'sp_message_iframe')]")

    # Prüfen, ob der Cookie-Banner in einem Iframe steckt
    if iframes:
        print("Banner liegt im iFrame")
        driver.switch_to.frame(iframes[0])

    cookie_button = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//button[@title='Zustimmen und weiter']")) 
    )
    cookie_button.click()
   
    if iframes:
        driver.switch_to.default_content()
        
    time.sleep(2) # Kurz warten, bis der Banner verschwindet

except TimeoutException:
    print("Kein Cookie-Banner gefunden")
    driver.switch_to.default_content()
    
##---------------------HTML Seite Speichern------------------------------------------------

target_date = datetime.strptime("08.09.2025", "%d.%m.%Y") #Ziel-Datum

page_num = 1
stop_found = False
base_filename = f"{datetime.now().strftime('%Y-%m-%d')}-zeit-seite"
print(f"Verarbeite Seite {page_num}")

current_html = driver.page_source


try:
    with open(base_filename, "w", encoding="utf-8") as f:
        f.write(current_html)
        print(f" Gespeichert: {base_filename}")

except Exception as e:
    print("HTML-Seite wurde nicht exportiert")

while True:
        
        ##------Artikel-Datum selektieren-----###
        articles = driver.find_elements(By.CLASS_NAME,"zon-teaser.zon-teaser--standard")
        last_article = articles[-1] #Den letzten Artikel selektieren
        time_stamp = last_article.find_element(By.TAG_NAME,"time")
        print(time_stamp)
        
        ##------Artikel-Datum in Datetime formatieren-----###
        date_str_value = time_stamp.get_attribute("datetime")
        date_object = datetime.fromisoformat(date_str_value)
        date_object = date_object.replace(tzinfo=None)
        date_value = date_object.strftime("%d.%m.%Y")
        print(f"date_value:{date_value}")

        ##------Navigieren der Seiten-----###
        if date_object > target_date:    
            
            try:
                
                file_name = os.path.join(Zeit_STOREAGE_DIR, f"{base_filename}-{page_num}.html")
                
                raw_html = driver.page_source
                
                with open(file_name, "w", encoding="utf-8") as f:
                    f.write(raw_html)
                    print(f" Gespeichert: {file_name}")
                page_num = page_num + 1
            except Exception as ex:
                print("Letzte Seite erreicht")
            
            soup = BeautifulSoup(current_html, "html.parser")
            wait = WebDriverWait(driver, 10)
            next_button = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "z-button.z-button--primary")))
            
            driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
            time.sleep(3)
            click_next_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME,'z-button.z-button--primary')))
            driver.execute_script("arguments[0].click();", click_next_button)

            WebDriverWait(driver, 10)
            time.sleep(2)
            
        else:
                
            file_name = os.path.join(Zeit_STOREAGE_DIR, f"{base_filename}-{page_num}.html")
            raw_html = driver.page_source
            
            with open(file_name, "w", encoding="utf-8") as f:
                f.write(raw_html)
                print(f" Gespeichert: {file_name}")
            print("Letzte Seite erreicht")
            driver.quit()
            break

Banner liegt im iFrame
Verarbeite Seite 1
 Gespeichert: 2026-01-30-zeit-seite
<selenium.webdriver.remote.webelement.WebElement (session="9d44d7bb7ead2a3519c3ff6c39e5dc13", element="f.7BA8B56ED4346098110526C9DD40BAEF.d.C1CB35CC2FF20F36678653C7E268FA39.e.34")>
date_value:31.12.2025
 Gespeichert: ../input/zeit_html-seiten/2026-01-30-zeit-seite-1.html
<selenium.webdriver.remote.webelement.WebElement (session="9d44d7bb7ead2a3519c3ff6c39e5dc13", element="f.7BA8B56ED4346098110526C9DD40BAEF.d.F12700A5BE19C75FB18A8635B15242D1.e.102")>
date_value:04.12.2025
 Gespeichert: ../input/zeit_html-seiten/2026-01-30-zeit-seite-2.html
<selenium.webdriver.remote.webelement.WebElement (session="9d44d7bb7ead2a3519c3ff6c39e5dc13", element="f.7BA8B56ED4346098110526C9DD40BAEF.d.F2A5A921229385ECBC120D083813403F.e.182")>
date_value:30.10.2025
 Gespeichert: ../input/zeit_html-seiten/2026-01-30-zeit-seite-3.html
<selenium.webdriver.remote.webelement.WebElement (session="9d44d7bb7ead2a3519c3ff6c39e5dc13", element="f

### HTML-Seite iterieren

In [39]:
zeit_article_id = []
zeit_title = []
zeit_content_preview = []
zeit_datum = []


seite = 1

page_num = page_num+1

zeit_filename = f"{datetime.now().strftime('%Y-%m-%d')}-zeit-seite"
while seite < page_num:    
    
    zeit_html = os.path.join(f"../input/zeit_html-seiten/{zeit_filename}-{seite}.html")
    print(zeit_html)
    
    
    with open(zeit_html, "r", encoding= "utf-8") as f:
        zeit_text = f.read()
    
    zeit_soup = BeautifulSoup(zeit_text, "html.parser") 
    zeit_article_tag = zeit_soup.select("article.zon-teaser.zon-teaser--standard")
    print()

    for zeit_article in zeit_article_tag:
        
        random_id = uuid.uuid4()
        zeit_article_id.append(random_id)
    
        #Titel des Artikels
    #------------------------------------------------------------------------------------------------------    
        zeit_title_tag = zeit_article.select('span.zon-teaser__title')
    
        if zeit_title_tag:
    
            for t in zeit_title_tag:
            
                zeit_title.append(t.get_text(strip=True))
                        
        else: 
            zeit_title.append('NA')
    
        # Vorschau des Artikel Inhaltes
    #------------------------------------------------------------------------------------------------------    
        zeit_preview_tag = zeit_article.select('p.zon-teaser__summary')
    
        if zeit_preview_tag:
            for p in zeit_preview_tag:
            
                zeit_content_preview.append(p.get_text(strip=True))
    
        else:
            zeit_content_preview.append('NA')
    
        # Datum der Veröffentlichung
    #------------------------------------------------------------------------------------------------------    
        zeit_datum_tag = zeit_article.select('time.zon-teaser__datetime')
    
        
        if zeit_datum_tag:
    
            for d in zeit_datum_tag:
    
                date_str_value = d.get("datetime")
                date_object = datetime.fromisoformat(date_str_value)
                zeit_datum = date_object.strftime("%d.%m.%Y")
        else:
            zeit_datum.append('NA')

    seite = seite +1
    
#print(f"Gefundene Artikel: {len(article_id)}") # Geändert
print(zeit_title)
print(zeit_content_preview)
print(zeit_datum)

../input/zeit_html-seiten/2026-01-30-zeit-seite-1.html

../input/zeit_html-seiten/2026-01-30-zeit-seite-2.html

../input/zeit_html-seiten/2026-01-30-zeit-seite-3.html

../input/zeit_html-seiten/2026-01-30-zeit-seite-4.html

../input/zeit_html-seiten/2026-01-30-zeit-seite-5.html

../input/zeit_html-seiten/2026-01-30-zeit-seite-6.html

../input/zeit_html-seiten/2026-01-30-zeit-seite-7.html

../input/zeit_html-seiten/2026-01-30-zeit-seite-8.html

['Südafrika weist ranghöchsten israelischen Diplomaten aus', 'Israel plant begrenzte Öffnung des Grenzübergangs Rafah für Sonntag', 'Radikale Siedler, gefördert von Israels Regierung', 'Zwischen Wunschvorstellung und Wirklichkeit', 'Israels Armee meldet Fund letzter toter Hamas-Geisel', 'Kommission in Israel findet Mängel bei Beschaffung deutscher U-Boote', 'Israels Einfluss auf die USA schrumpft', 'Sie brüllen sich gegenseitig als "Faschos" an', 'Plötzlich hat Netanjahu zum Iran nichts zu sagen', 'Die Zeugin, die Christina Block am nächsten stan

In [41]:
zeit_article_dict = {
    'id': zeit_article_id,
    'title': zeit_title,
    'preview': zeit_content_preview,
    'date': zeit_datum
}

In [43]:
zeit_df = pd.DataFrame(zeit_article_dict)
zeit_df["zeitschrift"] = "zeit"
zeit_df

,id,title,preview,date,zeitschrift
0,dd1df819-e8e1-4ec7-8dd8-9be9d66bdb00,Südafrika weist ranghöchsten israelischen Dipl...,Israels stellvertretender Botschafter soll bin...,05.09.2025,zeit
1,d9ccee28-a5f3-46d1-88f0-5e53f9dc6cf3,Israel plant begrenzte Öffnung des Grenzüberga...,Am Sonntag soll eine begrenzte Zahl von Mensch...,05.09.2025,zeit
2,51ee2898-3df2-4b46-b17a-f2e24b216b1d,"Radikale Siedler, gefördert von Israels Regierung",Organisierter und gewalttätiger: Die Angriffe ...,05.09.2025,zeit
3,02ff6fb1-ad38-4e2f-915f-f124685c38e4,Zwischen Wunschvorstellung und Wirklichkeit,Die letzte tote Geisel ist aus Gaza geborgen. ...,05.09.2025,zeit
4,20277e7d-3a78-4aaf-80be-9f0c3a1ecb0e,Israels Armee meldet Fund letzter toter Hamas-...,Die israelische Armee hat die letzte verbliebe...,05.09.2025,zeit
...,...,...,...,...,...
361,080c9db0-16aa-4610-a183-26f6472e33b2,Israelischer Flughafen von Drohne aus dem Jeme...,Nach dem Einschlag einer Drohne wurde der Betr...,05.09.2025,zeit
362,3aa37a42-5473-418d-afb1-eb7c7cdc82bc,Wie blicken die Menschen in Israel auf das Lei...,Israel steht international unter Druck wegen G...,05.09.2025,zeit
363,ad8ae049-8215-429d-97b9-8947ae52fcb3,"Israelisches Radteam entfernt ""Israel""-Schrift...",Die elfte Etappe der Vuelta musste wegen propa...,05.09.2025,zeit
364,92821f0b-0c0d-4e8f-b3ea-60a3bad3e2ea,Wo Großisrael schon sichtbar wird,Israel setzt seine Landnahme im Westjordanland...,05.09.2025,zeit


## Spiegel

In [45]:
def datum_tranformation(datum_text):

    monate = {
        "Januar": "01", "Februar": "02", "März": "03", "April": "04", 
        "Mai": "05", "Juni": "06", "Juli": "07", "August": "08", 
        "September": "09", "Oktober": "10", "November": "11", "Dezember": "12"
    }
    
    if "," in datum_text:
        datum_text = datum_text.split(",")[0]
        
    datum_text = datum_text.strip()
    
    for name, nummer in monate.items():
        if name in datum_text:
            datum_text = datum_text.replace(name, nummer)
            break
            
    datum_objekt = datetime.strptime(datum_text, "%d. %m %Y")
    
    return datum_objekt.strftime("%d.%m.%Y")

### Navigation der HTML-Seite

In [57]:
spiegel = "https://www.spiegel.de/thema/nahost/"
name = "spiegel"

driver = webdriver.Chrome()
driver.get(spiegel)

#---------------------Cookie-Button anklicken------------------------------------------------
try:
    wait = WebDriverWait(driver, 10)

    iframe = wait.until(EC.presence_of_element_located((By.XPATH, "//iframe[contains(@id, 'sp_message_iframe')]")))
    print("Iframe gefunden.")

    driver.switch_to.frame(iframe)
    print("Wechsel zu Iframe")

    cookie_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@title, 'Einwilligen und weiter')]")))
    
    cookie_button.click()
    print("Cookie Akzeptiert.")
    
    driver.switch_to.default_content()
    time.sleep(2)

except TimeoutException:
    print("Kein Banner gefunden oder Fehler beim Klicken.")
    driver.switch_to.default_content()
    
##---------------------HTML Seite Speichern------------------------------------------------

target_date = datetime.strptime("08.09.2025", "%d.%m.%Y") #Ziel-Datum

spiegel_page_num = 1
base_filename = f"{datetime.now().strftime('%Y-%m-%d')}-spiegel-seite"
print(f"Verarbeite Seite {spiegel_page_num}")

current_html = driver.page_source


try:
    with open(base_filename, "w", encoding="utf-8") as f:
        f.write(current_html)
        print(f" Gespeichert: {base_filename}")

except Exception as e:
    print("HTML-Seite wurde nicht exportiert")

while True:
        
        ##------Artikel-Datum selektieren-----###
        articles = driver.find_elements(By.CSS_SELECTOR, "article[data-target-teaser]")
        last_article = articles[-1]

        # Datum des letzten Artikel formatieren
        time_stamp_element = last_article.find_element(By.CSS_SELECTOR, "span[data-auxiliary]")
        time_stamp_text = time_stamp_element.text
        date_str_value = datum_tranformation(time_stamp_text)
        
        date_object = datetime.strptime(date_str_value, "%d.%m.%Y")
        print(date_object)
        

        ##------Navigieren der Seiten-----###
        if date_object > target_date:
            
            try:
                
                file_name = os.path.join(SPIEGEL_STOREAGE_DIR, f"{base_filename}-{spiegel_page_num}.html")
                
                raw_html = driver.page_source
                
                with open(file_name, "w", encoding="utf-8") as f:
                    f.write(raw_html)
                    print(f" Gespeichert: {file_name}")
                spiegel_page_num = spiegel_page_num + 1
            except Exception as ex:
                print("Letzte Seite erreicht")
            
            soup = BeautifulSoup(current_html, "html.parser")
            wait = WebDriverWait(driver, 10)
            next_button = wait.until(
            EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Ältere Artikel')]")))
            
            driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
            time.sleep(3)
            click_next_button = wait.until(EC.element_to_be_clickable((By.XPATH,"//span[contains(text(), 'Ältere Artikel')]")))
            driver.execute_script("arguments[0].click();", click_next_button)

            WebDriverWait(driver, 10)
            time.sleep(2)
            
        else:
                
            file_name = os.path.join(SPIEGEL_STOREAGE_DIR, f"{base_filename}-{spiegel_page_num}.html")
            raw_html = driver.page_source
            
            with open(file_name, "w", encoding="utf-8") as f:
                f.write(raw_html)
                print(f" Gespeichert: {file_name}")
            print("Letzte Seite erreicht")
            driver.quit()
            break

Iframe gefunden.
Wechsel zu Iframe
Cookie Akzeptiert.
Verarbeite Seite 1
 Gespeichert: 2026-01-30-spiegel-seite
2025-10-13 00:00:00
 Gespeichert: ../input/spiegel_html-seiten/2026-01-30-spiegel-seite-1.html
2025-07-27 00:00:00
 Gespeichert: ../input/spiegel_html-seiten/2026-01-30-spiegel-seite-2.html
Letzte Seite erreicht


### Daten Abfragen

In [71]:
spiegel_article_id = []
spiegel_title = []
spiegel_content_preview = []
spiegel_datum = []


seite = 1

spiegel_page_num = 2

spiegel_filename = f"{datetime.now().strftime('%Y-%m-%d')}-spiegel-seite"
while seite < spiegel_page_num:    
    
    spiegel_html = os.path.join(f"../input/spiegel_html-seiten/{spiegel_filename}-{seite}.html")
    print(spiegel_html)
    
    
    with open(spiegel_html, "r", encoding= "utf-8") as f:
        spiegel_text = f.read()
    
    spiegel_soup = BeautifulSoup(spiegel_text, "html.parser")
    #spiegel_article_tag = spiegel_soup.select("article")
    

    spiegel_article_tag = spiegel_soup.select("article")

    for spiegel_article in spiegel_article_tag:
        
        
        is_paid_content = spiegel_article.select_one("[data-flag-name='plus-paid'], #l-splus-flag-spon-spon-paid-flag-l")
        
        if is_paid_content:
        #Wenn gefunden: Es ist Spiegel+, also überspringen!
            continue
        
        
    
        id_tag = spiegel_article.attrs.get("data-sara-article-id", "KEINE ID GEFUNDEN")
        spiegel_article_id.append(id_tag)
        
#for zeit_article in zeit_article_tag:
    
 #   random_id = uuid.uuid4()
  #  spiegel_article_id.append(random_id)

    #Titel des Artikels
#------------------------------------------------------------------------------------------------------    
        spiegel_title_tag = spiegel_article.select("span.align-middle")
        #print(spiegel_title_tag)
    
        if spiegel_title_tag:
    
            for t in spiegel_title_tag:
                if t == "":
                    continue
                    
                titel_inhalt = t.get_text(strip=True)
                
                if not titel_inhalt: 
                    continue

                spiegel_title.append(titel_inhalt)   
                        
        else: 
            spiegel_title.append('NA')
    
        
    
        # Vorschau des Artikel Inhaltes
    #------------------------------------------------------------------------------------------------------    
        spiegel_preview_tag = spiegel_article.select("span.font-serifUI")
    
        if spiegel_preview_tag:
            for p in spiegel_preview_tag:
            
                spiegel_content_preview.append(p.get_text(strip=True))
    
        else:
            spiegel_content_preview.append('NA')
    
        # Datum der Veröffentlichung
    #------------------------------------------------------------------------------------------------------    
        spiegel_datum_tag = spiegel_article.select_one("span[data-auxiliary]")
    
        
        if spiegel_datum_tag:
    
            for d in spiegel_datum_tag:

                time_stamp_text = d.get_text(strip=True)
                date_str_value = datum_tranformation(time_stamp_text)
                date_object = datetime.strptime(date_str_value, "%d.%m.%Y")
    
                spiegel_datum.append(date_object)
    
                #date_str_value = d.get("datetime")
                #date_object = datetime.fromisoformat(date_str_value)
                #zeit_datum = date_object.strftime("%d.%m.%Y")
        else:
            spiegel_datum.append('NA')

    seite = seite +1

#print(f"Gefundene Artikel: {len(article_id)}") # Geändert
print(spiegel_article_id)
print(spiegel_title)
print(spiegel_content_preview)
print(spiegel_datum)

../input/spiegel_html-seiten/2026-01-30-spiegel-seite-1.html
['f80f7dfd-6a1d-461b-9135-96b6cb8b1dd2', '5f545db4-ccbd-4ad4-9489-2ef68f968854', '3e902700-9b73-44ae-a614-1f3c0a2db867', 'efcdba5f-9356-48fc-bd1d-1d9504106f51', '11eb8c72-bbc9-488b-a235-8533e829928b', 'e5057584-3f28-47ea-a1b7-06faa002cb44', 'ca24d03e-7478-42ca-9e0a-2abeaf987ad2', '3ac84165-8eea-48c9-b4d3-a1327f379bb4', 'f3e59609-a0d8-4788-9273-552fdc323794', '973c67b5-9cbd-4bbd-9148-5327d398ee9e', '1d8ea15b-4bac-4049-bdcc-02f5771bb86b', 'b2179a09-f264-4c1d-ae60-39fcdb1a5226']
['»Ich habe Freunde in Iran, von denen ich nicht mehr höre«', '»Ich habe Freunde in Iran, von denen ich nicht mehr höre«', 'Hass wie noch nie – wenn Linke gegen Linke auf die Straße gehen', 'Donald Trump lädt China zu Gaza-»Friedensrat« ein', 'Israelische Bagger zerstören die UNRWA-Zentrale', 'Papst Leo XIV. mahnt zu Frieden und erinnert die Gläubigen an ihre Verantwortung', 'Netanyahu will Milliarden in Rüstungsindustrie investieren', 'Libanons Präsiden

In [73]:
spiegel_article_dict = {
    'id': spiegel_article_id,
    'title': spiegel_title,
    'preview': spiegel_content_preview,
    'date': spiegel_datum
}

In [75]:
spiegel_df = pd.DataFrame(spiegel_article_dict)
spiegel_df["zeitschrift"] = "spielgel"
spiegel_df

,id,title,preview,date,zeitschrift
0,f80f7dfd-6a1d-461b-9135-96b6cb8b1dd2,"»Ich habe Freunde in Iran, von denen ich nicht...",Kontakte nach Iran sind wegen der Internetsper...,2026-01-21,spielgel
1,5f545db4-ccbd-4ad4-9489-2ef68f968854,"»Ich habe Freunde in Iran, von denen ich nicht...",Kontakte nach Iran sind wegen der Internetsper...,2026-01-21,spielgel
2,3e902700-9b73-44ae-a614-1f3c0a2db867,Hass wie noch nie – wenn Linke gegen Linke auf...,Leipzig-Connewitz ist eine Hochburg der Linken...,2026-01-20,spielgel
3,efcdba5f-9356-48fc-bd1d-1d9504106f51,Donald Trump lädt China zu Gaza-»Friedensrat« ein,Das chinesische Außenministerium hat eine Einl...,2026-01-20,spielgel
4,11eb8c72-bbc9-488b-a235-8533e829928b,Israelische Bagger zerstören die UNRWA-Zentrale,Israel geht seit Langem gegen das Uno-Palästin...,2026-01-20,spielgel
5,e5057584-3f28-47ea-a1b7-06faa002cb44,Papst Leo XIV. mahnt zu Frieden und erinnert d...,»Urbi et Orbi«: Bei seinem ersten Weihnachten ...,2025-12-25,spielgel
6,ca24d03e-7478-42ca-9e0a-2abeaf987ad2,Netanyahu will Milliarden in Rüstungsindustrie...,Israels Regierungschef Netanyahu will sein Lan...,2025-12-24,spielgel
7,3ac84165-8eea-48c9-b4d3-a1327f379bb4,Libanons Präsident wirbt bei Papst für Erhalt ...,Leo XIV. macht die zweite Station auf seiner e...,2025-11-30,spielgel
8,f3e59609-a0d8-4788-9273-552fdc323794,Rektorin der Uni Halle bedauert Genehmigung ei...,"»Einseitig, unwissenschaftlich und gesellschaf...",2025-10-30,spielgel
9,973c67b5-9cbd-4bbd-9148-5327d398ee9e,Prien fände ESC-Ausschluss Israels »grundfalsch«,Bildungsministerin Karin Prien befürwortet ein...,2025-10-30,spielgel


## Export

In [102]:
all_article_df = [sz_df, zeit_df, spiegel_df]
df_masterfile = pd.concat(all_article_df, ignore_index=True)

In [104]:
df_masterfile.to_csv(os.path.join(OUTPUT_DIR,"df_masterfile.csv"), sep=",")